# Computer Vision Project 3

## 1. Hough Transform for straight lines without edge orientation

### Binary Edge Detection

In [ ]:
import numpy as np
import cv2  
import matplotlib.pyplot as plt 

In [ ]:
def convolution(f, I):
    f_height, f_width = f.shape
    I_height, I_width = I.shape
    pad_height, pad_width = f_height // 2, f_width // 2

    I_padded = np.pad(I, ((pad_height, pad_height), (pad_width, pad_width)), mode='constant')

    im_conv = np.zeros(I.shape, dtype=np.float32)

    for i in range(I_height):
        for j in range(I_width):
            im_conv[i, j] = np.sum(f * I_padded[i:i + f_height, j:j + f_width])

    return im_conv

In [ ]:
image = cv2.imread('edges-lines-orig.png', cv2.IMREAD_GRAYSCALE)
plt.imshow(image, cmap='gray')

In [ ]:
sigma = 3
x_range = np.linspace(-int(sigma/2),int(sigma/2),sigma)
# print(x_range)
gaussian_filter = [ (1 / (sigma * np.sqrt(2*np.pi)) * np.exp(-x**2/(2*sigma**2))) for x in x_range ]
total = sum(gaussian_filter)
gaussian_filter = [[x/total for x in gaussian_filter]]
Gx = np.array(gaussian_filter)
Gy = Gx.reshape(-1,1)
print("Gx =",Gx)
print("Gy =",Gy)

In [ ]:
image_filtered_x = convolution(Gx, image)
image_filtered_y = convolution(Gy, image_filtered_x)

In [ ]:
plt.imshow(image_filtered_x, cmap='gray')

In [ ]:
plt.imshow(image_filtered_y, cmap='gray')

In [ ]:
derivative_filter_x = np.array([[-1], [0], [1]])
derivative_filter_y = derivative_filter_x.reshape((1, -1))

image_dx = convolution(derivative_filter_x, image_filtered_y)
image_dy = convolution(derivative_filter_y, image_filtered_y)


In [ ]:
plt.figure(figsize=(12, 12))

plt.subplot(1, 2, 1)
plt.imshow(image_dx, cmap='gray')
plt.title('Dervative Image Dx')

plt.subplot(1, 2, 2)
plt.imshow(image_dy, cmap='gray')
plt.title('Derivative Image Dy')

plt.show()

In [ ]:
gradient_magnitude = np.sqrt(image_dx ** 2 + image_dy ** 2)
plt.imshow(gradient_magnitude, cmap='gray')

In [ ]:
threshold = 30
binary_edges = (gradient_magnitude > threshold).astype(np.uint8) * 255

In [ ]:
cv2.imshow('Original Image', image.astype(np.uint8))
cv2.imshow('Gradient Magnitude', gradient_magnitude.astype(np.uint8))
cv2.imshow('Binary Edges', binary_edges)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
plt.figure(figsize=(12, 12))
plt.subplot(1, 3, 1)
plt.imshow(image, cmap='gray')
plt.title('Original Image')

plt.subplot(1, 3, 2)
plt.imshow(gradient_magnitude, cmap='gray')
plt.title('Gradient Magnitude')

plt.subplot(1, 3, 3)
plt.imshow(binary_edges, cmap='gray')
plt.title('Binary Edges(Threshold=40)')

plt.show()

In [ ]:
binary_edges = binary_edges[5:-5, 5:-5]

plt.imshow(binary_edges, cmap='gray')
plt.title('Binary Edges(Threshold=40)')

In [ ]:
def hough_transform(image, delta_rho=1, delta_theta=1, threshold=10, num_peaks=10):
    edges = binary_edges
    plt.imshow(edges, cmap='gray')

    max_rho = int(np.sqrt(np.square(image.shape[0]) + np.square(image.shape[1])))
    theta_values = np.deg2rad(np.arange(-90, 91, delta_theta))
    rho_values = np.arange(-max_rho, max_rho + 1, delta_rho)

    accumulator = np.zeros((len(rho_values), len(theta_values)), dtype=np.uint64)

    y_nonz, x_nonz = np.nonzero(edges)
    for i in range(len(x_nonz)):
        x = x_nonz[i]
        y = y_nonz[i]
        for j in range(len(theta_values)):
            rho = int(x * np.cos(theta_values[j]) + y * np.sin(theta_values[j]))
            rho_ind = np.argmin(np.abs(rho_values - rho))
            accumulator[rho_ind, j] += 1

    peaks = []

    for i in range(1, accumulator.shape[0] - 1):
        for j in range(1, accumulator.shape[1] - 1):
            if accumulator[i, j] > accumulator[i - 1:i + 2, j - 1:j + 2].max():
                peaks.append((i, j))

    peaks = sorted(peaks, key=lambda x: accumulator[x[0], x[1]], reverse=True)[:num_peaks]


    peaks = np.column_stack(np.unravel_index(np.argsort(accumulator.ravel())[-num_peaks:], accumulator.shape))

    lines_image = image.copy()
    for peak in peaks:
        rho, theta = rho_values[peak[0]], theta_values[peak[1]]
        a = np.cos(theta)
        b = np.sin(theta)
        x0 = a * rho
        y0 = b * rho
        x1 = int(x0 + 1000 * (-b))
        y1 = int(y0 + 1000 * (a))
        x2 = int(x0 - 1000 * (-b))
        y2 = int(y0 - 1000 * (a))
        cv2.line(lines_image, (x1, y1), (x2, y2), (0, 0, 255), 1)

    return accumulator, peaks, lines_image

In [ ]:
accumulator, peaks, lines_image = hough_transform(image)

# cv2.imshow('Original Image', image)
# cv2.imshow('Accumulator', cv2.normalize(accumulator, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8))
# cv2.imshow('Lines Image', lines_image)
# cv2.imshow('Binary Edges', binary_edges)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

In [ ]:
plt.figure(figsize=(12, 12))
plt.subplot(1, 3, 1)
plt.imshow(image, cmap='gray')
plt.title('Original Image')

plt.subplot(1, 3, 2)
plt.imshow(cv2.normalize(accumulator, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8), cmap='gray')
plt.title('Accumulator')

plt.subplot(1, 3, 3)
plt.imshow(lines_image, cmap='gray')
plt.title('Lines Image')

plt.show()

In [ ]:
normalized_accumulator = cv2.normalize(accumulator, None, 0, 255, cv2.NORM_MINMAX)

min_val = np.min(normalized_accumulator)
max_val = np.max(normalized_accumulator)
enhanced_accumulator = (normalized_accumulator - min_val) / (max_val - min_val) * 255

enhanced_accumulator = enhanced_accumulator.astype(np.uint8)

In [ ]:
# cv2.imshow('Original Image', image)
# cv2.imshow('Accumulator', cv2.normalize(accumulator, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8))
# cv2.imshow('Lines Image', lines_image)
# cv2.imshow('Binary Edges', binary_edges)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

## Creative Part - Hough Transform for Circles

In [ ]:
def hough_circle_transform(image, min_radius, max_radius, delta_radius=1, threshold=10, num_peaks=10):
    edges = cv2.Canny(image, 50, 150)

    max_radius_index = int((max_radius - min_radius) / delta_radius) + 1
    accumulator = np.zeros((image.shape[0], image.shape[1], max_radius_index), dtype=np.uint64)
    print("1")
    i=0
    edge_points = np.column_stack(np.nonzero(edges))
    for center_x in range(image.shape[1]):
        i+=1
        print("first",i)
        for center_y in range(image.shape[0]):
            for radius_index in range(max_radius_index):
                radius = min_radius + radius_index * delta_radius
                for point in edge_points:
                    x, y = point
                    if (x - center_x)**2 + (y - center_y)**2 == radius**2:
                        accumulator[center_y, center_x, radius_index] += 1
#     print("2")
    peaks = np.column_stack(np.unravel_index(np.argsort(accumulator.ravel())[-num_peaks:], accumulator.shape))
#     print("3")
    circles_image = image.copy()
    for peak in peaks:
        center_x, center_y, radius_index = peak
        radius = min_radius + radius_index * delta_radius
        cv2.circle(circles_image, (center_x, center_y), radius, (0, 0, 255), 2)
#     print("4")
    return accumulator, peaks, circles_image

In [ ]:
image = cv2.imread('edges-lines-orig.png', cv2.IMREAD_GRAYSCALE)
min_radius = 11
max_radius = 12
delta_radius = 1

accumulator, peaks, circles_image = hough_circle_transform(image, min_radius, max_radius, delta_radius)

# print("Done")


cv2.imshow('Original Image', image)
# cv2.imshow('Accumulator', cv2.normalize(accumulator, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8))
cv2.imshow('Circles Image', circles_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
plt.figure(figsize=(12, 12))
plt.subplot(1, 3, 1)
plt.imshow(image, cmap='gray')
plt.title('Original Image')

acc = cv2.normalize(accumulator, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)
plt.subplot(1, 3, 2)
plt.imshow(acc[:, :,0], cmap='gray')
plt.title('Accumulator')

plt.subplot(1, 3, 3)
plt.imshow(circles_image, cmap='gray')
plt.title('Lines Image')

plt.show()